In [64]:
import numpy as np
import random
from PIL import Image
import os
import re

In [65]:
# матрицу в вектор
def mat_to_vec(x):
    m = x.shape[0]*x.shape[1]
    tmp1 = np.zeros(m)

    c = 0
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            tmp1[c] = x[i,j]
            c +=1
    return tmp1


# создаем матрицу весов для одного изображения
def create_W(x):
    w = np.zeros([len(x),len(x)])
    for i in range(len(x)):
        for j in range(i,len(x)):
            if i == j:
                w[i,j] = 0 # обнуляем если номер строки и столбца совпадают, так получим обнуление по диагонали
            else:
                w[i,j] = x[i]*x[j]
                w[j,i] = w[i,j]
    return w

# Приводим изображение к виду состоящего из матрицы из единиц и минус единиц
def readImg2array(file,size, threshold = 145): # threshold - порог, чтобы преобразовать все значения канала выше него в 1
    pilIN = Image.open(file).convert(mode="L")
    pilIN = pilIN.resize(size)

    imgArray = np.asarray(pilIN,dtype=np.uint8)
    # print("imgArray", imgArray, "imgArray.shape", imgArray.shape)
    x = np.zeros(imgArray.shape,dtype=np.float)
    x[imgArray > threshold] = 1 # соответственно здесь и нужен этот порог, все что выше него становится 1
    x[x==0] = -1 # все что ниже порога становится -1 (так получим контрастное изображение)
    # print("Зашумленная матрица преобразованная \n", x)
    return x

# Из вектора обратно в изображение
def array2img(data, outFile = None, show = True):
    
    # инициализируем матрицу из нулей
    y = np.zeros(data.shape,dtype=np.uint8)
    # преобразуем данные из 1 и -1 в значения канала
    y[data==1] = 255
    y[data==-1] = 0
    print("Матрица в формате изображения — для наглядности изменений:\n", y)
    img = Image.fromarray(y,mode="L")
    if show == True:
        img.show()
    if outFile is not None:
        img.save(outFile)
    return img

def array2colorCode(data, outFile = None):
    
    # инициализируем матрицу из нулей
    y = np.zeros(data.shape,dtype=np.uint8)
    # преобразуем данные из 1 и -1 в значения канала
    y[data==1] = 255
    y[data==-1] = 0

    return y


# обновляем веса - останова по количеству итераций
def update(w,y_vec,theta=0.5,time=1000):
    global t
    t = False
    while t == False:
        y_vec1 = y_vec
        for s in range(time):
            m = len(y_vec)
            i = random.randint(0,m-1) # случайным выбирает значение в пределах размера восстанавливаемого образца
            u = np.dot(w[i][:],y_vec) - theta # перемножаем случайный i-й столбец с вектором образца                                                                                                                - theta

            if u >= 0: # фунция активации
                y_vec[i] = 1
            elif u < 0:
                y_vec[i] = -1
        t = (y_vec==y_vec1).all()
#        print("Основной цикл обновления")
#        print("y_vec", y_vec.reshape(oshape))
#        print("y_vec1", y_vec1.reshape(oshape))
#        print("Так",t)

    for test in range(1, 4):
        
        m = len(y_vec)
        i = random.randint(0,m-1) # случайным выбирает значение в пределах размера восстанавливаемого образца
        u = np.dot(w[i][:],y_vec) - theta # перемножаем случайный i-й столбец с вектором образца                                                                                                                - theta
        
        if u >= 0: # фунция активации
            y_vec[i] = 1
        elif u < 0:
            y_vec[i] = -1
            
        print("Тест №:", test, "\n", y_vec.reshape(oshape), y_vec.shape, "\n")        
    print("Вектор восстановленный и вектор исходный равны, ура!", "Это тому подверждение - здесь функция сравнивает эти векторы и отдает соотвественно:", (y_vec==y_vec1).all())
    y_vec = y_vec.reshape(oshape)
#        print("Для себя", y_vec)
    
    return y_vec

## Обучение

In [73]:
def hopfield1(theta=0.5, time=1000, size=(100,100), threshold=60, current_path=None):
    
    # загрузка исходного образца
    # получение изображение и преобразование его в вектор
    print("Импорт изображения и создание матрицы весов...")
    
    path_train = "/Users/germankucheravenko/Documents/python/Интеграция ИТ в ИС/Hopfield/Hopfield_net/train_pics/2.jpg"
    print(path_train)
    x = readImg2array(file=path_train,size=size,threshold=threshold)
    print("Оригинальная матрица преобразованная\n", x)
    global x_vec
    x_vec = mat_to_vec(x)
    print("Длина получившегося вектора:", len(x_vec))
    
    w = create_W(x_vec)

    print("Матрица весов: \n", w)
    
    # Импорт восстанавливаемых изображений
    
    path_test = "/Users/germankucheravenko/Documents/python/Интеграция ИТ в ИС/Hopfield/Hopfield_net/test_pics/2_test.jpg"
    y = readImg2array(file=path_test,size=size,threshold=threshold) # преобразуем изображение в матрицу чисел
    print("Зашумленная матрица преобразованная\n", y)
    global oshape
    oshape = y.shape
    y_img = array2img(y)

    print("Тестовые данные загружены")

    y_vec = mat_to_vec(y) # преобразуем матрицу чисел в один длинный вектор строка за строкой
#    global y_vec1
#    y_vec1 = y_vec1
    print("Обновление...")
    
    y_vec_after = update(w=w,y_vec=y_vec,theta=theta,time=time)
    
    for test in range(1, 1):
        y_vec_after = update(w=w,y_vec=y_vec,theta=theta,time=time)
        print("Готовая матрица из единиц \n", y_vec_after, y_vec_after.shape, "\n")
    
    y_vec_after = y_vec_after.reshape(oshape) # из вектора обратно в матрицу
    
    
    if current_path is not None: # сохраняем восстановленное изображение в файл "after_*.jpg"
        outfile = current_path+"/after_"+".jpeg"
        array2img(y_vec_after,outFile=outfile)
    else:
        after_img = array2img(y_vec_after,outFile=None)
        after_img.show()
                
    # print(y_vec_after, y_vec_after.shape)
        
        
current_path = os.getcwd()

In [74]:
# Запуск
hopfield1(theta=0.5,time=20000,size=(16,16),threshold=60, current_path = current_path)

Импорт изображения и создание матрицы весов...
/Users/germankucheravenko/Documents/python/Интеграция ИТ в ИС/Hopfield/Hopfield_net/train_pics/2.jpg
Оригинальная матрица преобразованная
 [[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. 

## Для нескольких образцов

In [60]:
def hopfield(train_files, test_files,theta=0.5, time=1000, size=(100,100),threshold=60, current_path=None):
    
    # загрузка исходного образца
    # получение изображение и преобразование его в вектор
    print("Импорт изображения и создание матрицы весов...")

    # num_files - количество файлов
    num_files = 0
    for path in train_files:
        print(path)
        x = readImg2array(file=path,size=size,threshold=threshold)
        print("Оригинальная матрица преобразованная №", num_files+1, "\n", x)
        x_vec = mat_to_vec(x)
        print("Длина получившегося вектора:", len(x_vec))
        if num_files == 0:
            w = create_W(x_vec)
            num_files = 1
        else:
            tmp_w = create_W(x_vec)
            w = w + tmp_w
            num_files +=1

    print("Матрица весов: \n", w)


    # Импорт восстанавливаемых изображений
    counter = 0
    for path in test_files:
        y = readImg2array(file=path,size=size,threshold=threshold) # преобразуем изображение в матрицу чисел
        
        print("\n\nТестовые данные загружены")
        
        print("\nМатрица №", counter+1)
        print("Зашумленная матрица\n", y)
        # print(array2colorCode(y))
        oshape = y.shape
        y_img = array2img(y, show = False)
        y_img.show()

        y_vec = mat_to_vec(y) # преобразуем матрицу чисел в один длинный вектор строка за строкой
        # time = len(y_vec)
        print("Обновление...")
        
        y_vec_after = update(w=w,y_vec=y_vec,theta=theta,time=time)
        # print(y_vec_after, y_vec_after.shape)
        
        y_vec_after = y_vec_after.reshape(oshape) # из вектора обратно в матрицу
        # print(y_vec_after, y_vec_after.shape)
        
        if current_path is not None: # сохраняем восстановленное изображение в файл "after_*.jpg"
            outfile = current_path+"/after_"+str(counter+1)+".jpeg"
            array2img(y_vec_after,outFile=outfile)
        else:
            after_img = array2img(y_vec_after,outFile=None, show=False)
            after_img.show()
        counter +=1

# Исходные данные
current_path = os.getcwd()
train_paths = []
path = current_path+"/train_pics/"
for i in os.listdir(path):
    if re.match(r'[0-9a-zA-Z-]*.jp[e]*g',i):
        train_paths.append(path+i)

# Зашумленные
test_paths = []
path = current_path+"/test_pics/"
for i in os.listdir(path):
    if re.match(r'[0-9a-zA-Z-_]*.jp[e]*g',i):
        test_paths.append(path+i)

In [61]:
# Запуск
hopfield(train_files=train_paths, test_files=test_paths, theta=0.5,time=10000,size=(16,16),threshold=60, current_path = current_path)

Импорт изображения и создание матрицы весов...
/Users/germankucheravenko/Documents/python/Интеграция ИТ в ИС/Hopfield/Hopfield_net/train_pics/2.jpg
Оригинальная матрица преобразованная № 1 
 [[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.

Тест №: 1 
 [[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 